In [1]:
!pip install -q py-feat
# !pip install -q facial-emotion-recognition
# !pip install -q fer
# !pip install -q deepface
# !pip install -q mtcnn

     |████████████████████████████████| 6.0MB 7.2MB/s 
     |████████████████████████████████| 3.3MB 28.8MB/s 
     |████████████████████████████████| 57.4MB 47kB/s 
     |████████████████████████████████| 4.9MB 39.1MB/s 


In [12]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
from feat import Detector
import glob
import math
import dlib
import joblib
import cv2
from tqdm.notebook import tqdm
from google.colab.patches import cv2_imshow
from collections import Counter

import tensorflow as tf
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.models import Model
from keras import optimizers 
from keras import regularizers
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau



def my_mode(sample):
  c = Counter(sample)
  return [k for k, v in c.items() if v == c.most_common(1)[0][1]]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
face_model = "RetinaFace"
landmark_model = "MobileNet"
au_model = "svm"
emotion_model = "resmasknet" #resmasknet,fer, svm, rf
detector = Detector(face_model = face_model, landmark_model = landmark_model, au_model = au_model, emotion_model = emotion_model)

Loading Face Detection model:  RetinaFace
Using downloaded and verified file: /usr/local/lib/python3.7/dist-packages/feat/resources/mobilenet0.25_Final.pth
Using downloaded and verified file: /usr/local/lib/python3.7/dist-packages/feat/resources/mobilenet_224_model_best_gdconv_external.pth.tar
Using downloaded and verified file: /usr/local/lib/python3.7/dist-packages/feat/resources/hog_pca_all_emotio.joblib
Using downloaded and verified file: /usr/local/lib/python3.7/dist-packages/feat/resources/hog_pca_all_emotio.joblib
Using downloaded and verified file: /usr/local/lib/python3.7/dist-packages/feat/resources/hog_scalar_aus.joblib
Using downloaded and verified file: /usr/local/lib/python3.7/dist-packages/feat/resources/svm_568.joblib
Using downloaded and verified file: /usr/local/lib/python3.7/dist-packages/feat/resources/hog_pca_all_emotio.joblib
Using downloaded and verified file: /usr/local/lib/python3.7/dist-packages/feat/resources/hog_scalar_aus.joblib
Using downloaded and verifie

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PCA from version 0.24.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearSVC from version 0.24.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [ ]:
pred_emotion= []

filepath = 'drive/MyDrive/file2/done/face9_5.csv'
vdopath = 'drive/MyDrive/file2/done/face9_5.mp4'
df = pd.read_csv(filepath)
dim = (224, 224)
rowimg = []
label_img = []
print('df', df.shape)

for i in tqdm(range(df.shape[0])):
  seqimg = []
  for j in range(df['start_frame'][i], df['end_frame'][i]):
    cap = cv2.VideoCapture(vdopath)
    cap.set(cv2.CAP_PROP_POS_FRAMES, j)
    success, image = cap.read()
    # try:
    preds = detector.detect_image(image).emotions()
    pred_class = np.argmax(preds)
    seqimg.append(pred_class) 
    # except:
    #   print("Emotion Error  i = ",i, ' j =', j)

  seq_label = my_mode(seqimg)
  try:
    pred_emotion.append(seq_label[0])
  except:
    pass

df (22, 7)


In [ ]:
len(pred_emotion)

In [ ]:
df = pd.DataFrame(np.array(pred_emotion), columns=['label'])
from google.colab import files
df.to_csv('face9_5_r.csv') 
files.download('face9_5_r.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
while True:pass